# Project - Building an Image Narration Tool

---

## Approach 1: Using Hugging Face Transformers pipelines

### Step 1: Object Detection from Image

In [ ]:
! pip install transformers torch pillow requests tqdm iprogress

In [1]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

# you can specify the revision tag if you don't want the timm dependency
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.9
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )


D:\ai\project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████████████████████████████████| 530/530 [00:00<00:00, 858.79it/s, Materializing param=model.query_position_embeddings.weight]


Detected remote with confidence 0.998 at location [40.16, 70.81, 175.55, 117.98]
Detected remote with confidence 0.996 at location [333.24, 72.55, 368.33, 187.66]
Detected couch with confidence 0.995 at location [-0.02, 1.15, 639.73, 473.76]
Detected cat with confidence 0.999 at location [13.24, 52.05, 314.02, 470.93]
Detected cat with confidence 0.999 at location [345.4, 23.85, 640.37, 368.72]


Exception in thread Thread-auto_conversion:
Traceback (most recent call last):
  File "C:\Users\smadduri\AppData\Roaming\uv\python\cpython-3.14.2-windows-x86_64-none\Lib\threading.py", line 1082, in _bootstrap_inner
    self._context.run(self.run)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^
  File "C:\Users\smadduri\AppData\Roaming\uv\python\cpython-3.14.2-windows-x86_64-none\Lib\threading.py", line 1024, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\ai\project\.venv\Lib\site-packages\transformers\safetensors_conversion.py", line 117, in auto_conversion
    raise e
  File "D:\ai\project\.venv\Lib\site-packages\transformers\safetensors_conversion.py", line 96, in auto_conversion
    sha = get_conversion_pr_reference(api, pretrained_model_name_or_path, **cached_file_kwargs)
  File "D:\ai\project\.venv\Lib\site-packages\transformers\safetensors_conversion.py", line 77, in get_conversion_pr_reference
    raise OSError(
    ...<2 lines>...

### Step 2: Extract Labels and Convert to Text

In [2]:
from collections import Counter

def generate_natural_language(results, model):
    # 1. Extract the labels from your results tensor
    detected_labels = [model.config.id2label[label.item()] for label in results["labels"]]
    
    if not detected_labels:
        return "I scanned the image but didn't find any objects with high confidence."

    # 2. Count the occurrences of each object
    counts = Counter(detected_labels)
    
    # 3. Format into a list of strings (e.g., "2 cats")
    formatted_items = []
    for obj, count in counts.items():
        # Add an 's' for plurals (basic logic)
        label = f"{obj}s" if count > 1 else obj
        formatted_items.append(f"{count} {label}")

    # 4. Join them into a natural sentence
    if len(formatted_items) == 1:
        description = f"In this image, I can see {formatted_items[0]}."
    else:
        # Connects items with commas and an 'and' for the last item
        description = f"In this image, I can see {', '.join(formatted_items[:-1])} and {formatted_items[-1]}."
    
    return description

# Call the function using your existing 'results' and 'model' variables
text_summary = generate_natural_language(results, model)
print(text_summary)

In this image, I can see 2 remotes, 1 couch and 2 cats.


### Step 3: Text-to-Speech (TTS)

In [ ]:
! pip install transformers scipy

In [3]:
from transformers import AutoProcessor, AutoModel

processor = AutoProcessor.from_pretrained("suno/bark-small")
model = AutoModel.from_pretrained("suno/bark-small")

inputs = processor(
    text=[text_summary],
    return_tensors="pt",
)

speech_values = model.generate(**inputs, do_sample=True)


Loading weights: 100%|███████████████████████████████████████| 542/542 [00:00<00:00, 880.11it/s, Materializing param=semantic.position_embeds_layer.weight]
The tied weights mapping and config for this model specifies to tie fine_acoustics.input_embeds_layers.1.weight to fine_acoustics.lm_heads.0.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie fine_acoustics.input_embeds_layers.2.weight to fine_acoustics.lm_heads.1.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie fine_acoustics.input_embeds_layers.3.weight to fine_acoustics.lm_heads.2.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the confi

In [4]:
from IPython.display import Audio

sampling_rate = model.generation_config.sample_rate
Audio(speech_values.cpu().numpy().squeeze(), rate=sampling_rate)


---

## Approach 2: using Google GenAI library and HuggingFace Transformers pipeline

### Step 1: Image Captioning using Google GenAI SDK

In [ ]:
! pip install tiktoken google-genai

In [5]:
from google import genai
from google.genai import types

client = genai.Client(api_key="YOUR_API_KEY")


system_prompt = """
"You are a helpful AI Assistant. Given an image perform object detection and provide a text output which contains the information about the labels detected and their counts."
Always start your response with the exact phrase: "Here is what I see in the image:".
Make the output look a little more in english
"""

response = client.models.generate_content(
    model="gemini-3-flash-preview",
    config=types.GenerateContentConfig(
        system_instruction=system_prompt
    ),
    contents=f"Describe these objects: {text_summary}"
)

print(response.text)

Here is what I see in the image:

- There are **two cats** resting comfortably.
- I also see **two remotes** lying on the surface.
- They are all on **one couch**.


### Step 2: Text-to-Speech (TTS)

In [ ]:
! pip install scipy

In [6]:
from transformers import AutoProcessor, AutoModel

processor = AutoProcessor.from_pretrained("suno/bark-small")
model = AutoModel.from_pretrained("suno/bark-small")

inputs = processor(
    text=[response.text],
    return_tensors="pt",
)

speech_values = model.generate(**inputs, do_sample=True)


Loading weights: 100%|███████████████████████████████████████| 542/542 [00:00<00:00, 801.22it/s, Materializing param=semantic.position_embeds_layer.weight]
The tied weights mapping and config for this model specifies to tie fine_acoustics.input_embeds_layers.1.weight to fine_acoustics.lm_heads.0.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie fine_acoustics.input_embeds_layers.2.weight to fine_acoustics.lm_heads.1.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie fine_acoustics.input_embeds_layers.3.weight to fine_acoustics.lm_heads.2.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the confi

In [7]:
from IPython.display import Audio

sampling_rate = model.generation_config.sample_rate
Audio(speech_values.cpu().numpy().squeeze(), rate=sampling_rate)
